In [1]:
# Import modules
import osmnx as ox 

import numpy as np
import geopandas as gpd
import pandas as pd

from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import LineString, MultiLineString
# Import module
import networkx as nx
import matplotlib.pyplot as plt

from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon
ox.config(log_console=True, use_cache=True)
ox.__version__

'1.1.2'

2022-02-21 14:35:24 Configured OSMnx 1.1.2
2022-02-21 14:35:24 HTTP response caching is on


# limit Vax Data to 3 Counties Comprising Detroit Metro Area

In [2]:
vax_df = pd.read_csv('/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Covid-19_Vaccination_Provider_Locations_in_the_United_States/Covid-19_Vaccination_Provider_Locations_in_the_United_States.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Covid-19_Vaccination_Provider_Locations_in_the_United_States/Covid-19_Vaccination_Provider_Locations_in_the_United_States.csv'

In [16]:
# Convert DataFrame to GeoDataFrame
vax_gdf = gpd.GeoDataFrame(vax_df, geometry=gpd.points_from_xy(vax_df['X'], vax_df['Y']))

In [17]:
#Reproject to UTM 17
vax_gdf.crs = 'EPSG:32617'

I wonder what it would look like to adapt the following code to each vaccine clinic in Detroit and turn it into walk and driving from the lat lon with the set distance. 

In [18]:
#limit by state to Michigan
vax_MI = vax_gdf[vax_gdf['State']== 'MI'] 

In [19]:
vax_MI['county'].unique()

array(['Alpena', 'Kalamazoo', 'Montmorency', 'Marquette', 'Alcona',
       'Eaton', 'Genesee', 'Macomb', 'Oakland', 'Monroe', 'Saginaw',
       'Wayne', 'Washtenaw', 'Livingston', 'Ingham', 'Kent',
       'Grand Traverse', 'Berrien', 'Muskegon', 'Isabella', 'Emmet',
       'Calhoun', 'Bay', 'Ottawa', 'St. Clair', 'Menominee', 'Van Buren',
       'St. Joseph', 'Jackson', 'Ionia', 'Lenawee', 'Cheboygan',
       'Dickinson', 'Ogemaw', 'Iosco', 'Oscoda', 'Presque Isle',
       'Crawford', 'Kalkaska', 'Lake', 'Manistee', 'Mason', 'Mecosta',
       'Missaukee', 'Newaygo', 'Oceana', 'Wexford', 'Midland', 'Benzie',
       'Clinton', 'Gratiot', 'Montcalm', 'Otsego', 'Branch', 'Lapeer',
       'Antrim', 'Charlevoix', 'Hillsdale', 'Shiawassee', 'Delta',
       'Tuscola', 'Allegan', 'Cass', 'Sanilac', 'Gladwin', 'Osceola',
       'Clare', 'Roscommon', 'Arenac', 'Barry', 'Huron', 'Leelanau', nan],
      dtype=object)

In [21]:
#Limit By County to Wayne, Oakland, and Macomb
vax_wayne = vax_MI[vax_MI['county']== 'Wayne']
vax_oakland = vax_MI[vax_MI['county']== 'Oakland'] 
vax_macomb = vax_MI[vax_MI['county']== 'Macomb']

In [24]:
#Agglomerate to one GDF of Detroit Metro
vax_detr_metro = vax_wayne 
vax_detr_metro = vax_detr_metro.append(vax_oakland)
vax_detr_metro = vax_detr_metro.append(vax_macomb)

/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_43921/951935581.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vax_detr_metro = vax_detr_metro.append(vax_oakland)
/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_43921/951935581.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vax_detr_metro = vax_detr_metro.append(vax_macomb)


# OSMNX Isocrhone Maps

In [ ]:
# configure the place, network type, trip times, and travel speed
place = 'Berkeley, CA, USA'
network_type = 'walk'
trip_times = [5, 10, 15, 20, 25] #in minutes
travel_speed = 4.5 #walking speed in km/hour

In [ ]:
# download the street network
G = ox.graph_from_place(place, network_type=network_type)

In [ ]:
# find the centermost node and then project the graph to UTM
gdf_nodes = ox.graph_to_gdfs(G, edges=False)
x, y = gdf_nodes['geometry'].unary_union.centroid.xy
center_node = ox.get_nearest_node(G, (y[0], x[0]))
G = ox.project_graph(G)

In [ ]:
# add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60 #km per hour to m per minute
for u, v, k, data in G.edges(data=True, keys=True):
    data['time'] = data['length'] / meters_per_minute

In [ ]:
# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0, return_hex=True)

In [ ]:
# color the nodes according to isochrone then plot the street network
node_colors = {}
for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')
    for node in subgraph.nodes():
        node_colors[node] = color
nc = [node_colors[node] if node in node_colors else 'none' for node in G.nodes()]
ns = [15 if node in node_colors else 0 for node in G.nodes()]
fig, ax = ox.plot_graph(G, node_color=nc, node_size=ns, node_alpha=0.8, node_zorder=2,
                        bgcolor='k', edge_linewidth=0.2, edge_color='#999999')

In [ ]:
# make the isochrone polygons
isochrone_polys = []
for trip_time in sorted(trip_times, reverse=True):
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)

In [ ]:
# plot the network then add isochrones as colored descartes polygon patches
fig, ax = ox.plot_graph(G, show=False, close=False, edge_color='#999999', edge_alpha=0.2,
                        node_size=0, bgcolor='k')
for polygon, fc in zip(isochrone_polys, iso_colors):
    patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
    ax.add_patch(patch)
plt.show()

In [ ]:
def make_iso_polys(G, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')

        node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({'id': subgraph.nodes()}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index('id')

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get('geometry',  LineString([f,t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union
        
        # try to fill in surrounded areas so shapes will appear solid and blocks without white space inside them
        if infill:
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys.append(new_iso)
    return isochrone_polys

isochrone_polys = make_iso_polys(G, edge_buff=25, node_buff=0, infill=True)
fig, ax = ox.plot_graph(G, show=False, close=False, edge_color='#999999', edge_alpha=0.2,
                        node_size=0, bgcolor='k')
for polygon, fc in zip(isochrone_polys, iso_colors):
    patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
    ax.add_patch(patch)
plt.show()